# 8장 합성곱 신경망: 컴퓨터 비전

**감사말**: 프랑소와 숄레의 [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff) 8장에 사용된 코드에 대한 설명을 담고 있으며 텐서플로우 2.6 버전에서 작성되었습니다. 소스코드를 공개한 저자에게 감사드립니다.

**tensorflow 버전과 GPU 확인**
- 구글 코랩 설정: '런타임 -> 런타임 유형 변경' 메뉴에서 GPU 지정 후 아래 명령어 실행 결과 확인

    ```
    !nvidia-smi
    ```

- 사용되는 tensorflow 버전 확인

    ```python
    import tensorflow as tf
    tf.__version__
    ```
- tensorflow가 GPU를 사용하는지 여부 확인

    ```python
    tf.config.list_physical_devices('GPU')
    ```

## 주요 내용

- 합성곱 신경망(convnet) 이해
- convnet 재활용

# 8.1 합성곱 신경망 소개

### 예제: MNIST 데이터셋 분류

**convnet 모델 구성**

- `Input()`의 `shape`: `(28, 28, 1)`
- `Conv2D`와 `MaxPooling2D` 층을 함수형 API 방식으로 층쌓기
    - 출력값: 3D 텐서 (높이, 너비, 채널수)
    - 채널수: `filters` 인자로 결정
    - `kernel_size=3`에 유의할 것.

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

# 입력층
inputs = keras.Input(shape=(28, 28, 1))

# 은닉층
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)

x = layers.Flatten()(x) # 1차원 텐서로 변환

# 출력층
outputs = layers.Dense(10, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

**모델 구성 요약**

In [2]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0     

**MNIST 이미지 분류 훈련**

모델 훈련은 이전과 동일하다.

In [3]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

model.compile(optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 16s 17ms/step - loss: 0.1585 - accuracy: 0.9514
Epoch 2/5
938/938 [==============================] - 16s 17ms/step - loss: 0.0429 - accuracy: 0.9867
Epoch 3/5
938/938 [==============================] - 17s 18ms/step - loss: 0.0309 - accuracy: 0.9901
Epoch 4/5
938/938 [==============================] - 18s 19ms/step - loss: 0.0231 - accuracy: 0.9930
Epoch 5/5
938/938 [==============================] - 17s 19ms/step - loss: 0.0170 - accuracy: 0.9947


**훈련된 convnet 평가**

테스트셋에 대한 성능이 이전에 사용한 `Sequential` 모델보다 좋아졌다.

In [4]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

313/313 [==============================] - 1s 2ms/step - loss: 0.0251 - accuracy: 0.9925
Test accuracy: 0.993


### 합성곱 연산

- `Dense` 층: 입력값 전체를 학습 대상으로 삼음.
    - MNIST의 경우: 숫자 이미지 전체
- `Conv2D` 층: 예를 들어 `kernel_size=3`으로 설정된 경우
    `3x3` 크기의 국소적 특성만 대상으로 삼음.
    다음 두 가지 장점을 발휘함.

**Conv2D 모델의 장점 1: 위치 독립성**

한 번 인식한 패턴을 다른 위치에서 인실할 수 있다. 
적은 수의 샘플을 이용하여 좋은 성능의 일반화 모델 훈련 가능해진다.   

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/local_patterns.jpg" style="width:500px;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

**Conv2D 모델의 장점 2: 패턴 공간 계층 파악**

위 층으로 진행할 수록 보다 복잡한 패턴을 파악한다.
이를 **패턴 공간 계층**(spatial hierarchy of patterns)이라 부른다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/visual_hierarchy_hires.png" style="width:500px;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

### 특성맵, 채널, 필터

합성곱 연산의 작동법을 이해하려면 아래 세 개념을 이해해야 한다.

- **특성맵**(feature maps): 높이, 너비, 깊이로 구성된 3D 텐서
    - 예제: MNIST 샘플의 경우 `(28, 28, 1)` 모양의 3D 텐서.
- **채널**(channel): 특성맵의 깊이를 구성하는 값.
    - 예제: MNIST 데이터 샘플은 흑백 사진이기에 하나의 채널로 구성됨.
    - 예제: 컬러 이미지 샘플은 `(28, 28, 3)` 과 같은 모양의 3D 텐서이기에
        세 개의 채널로 구성됨. 3은 RGB를 나타냄.        
- **필터**(filter): `kernel_size`를 이용한 3D 텐서. 
    - 예제: `kernel_size=3`인 경우 필터는 `(3, 3, 입력특성맵깊이)` 모양의 3D 텐서.
    - 하나의 필터가 하나의 **출력맵**(response map), 즉 하나의 채널을 생성하는 데에 사용됨.
    - 필터 수: `filters` 인자에 의해 결정됨.
    - 출력 특성맵: `(높이, 너비, 필터수)` 모양의 3D 텐서

아래 그림은 MNIST 데이터 샘플에 대해 하나의 필터가 작동하여 하나의 출력맵을 생성하는 과정을 보여준다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/response_map_hires.png" style="width:500px;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

아래 그림은 세 개의 필터를 작동시켜 출력 특성맵을 생성하는 과정을 보여주며
사용된 예제는 다음과 같다.

- 입력 특성맵: `(5, 5, 2)` 모양의 3D 텐서
- 필터 모양: `(3, 3, 2)`
- 필터 개수: 3개
- 출력 특성맵: `(3, 3, 3)` 모양의 3D 텐서

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp/master/notebooks/images/how_convolution_works.jpg" style="width:500px;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

### 합성곱 커널과 합성곱 신경망

필터는 입력 특성맵을 출력 특성맵으로 변환하는 과정에서 사용되는 가중치 행렬이며,
이런 필터를 출력 특성맵의 깊이(채널 수)만큼 모아놓은 (4D) 텐서가 **합성곱 커널**이다.
**합성곱 신경망**은 바로 이 합성곱 커널(필터)을 학습시키는 모델을 가리킨다.

### 패딩과 보폭

입력 특성맵의 높이와 너비가 출력 특성맵의 높이와 너비랑 다를 수 있다.
이는 **패딩**(padding) 사용여부와 **보폭**(stride)의 크기에 의에 결정된다.
다음 세 개의 그림은 입력 특성맵의 높이와 너비가 `5x5`일 때 패딩 여부와 보폭의 크기에 
따라 출력 특성맵의 높이와 너비가 어떻게 달라지는가를 보여준다.

- 경우 1: 패딩 없음, 보폭은 1.
    - 출력 특성맵의 깊이와 너비: `3x3`
    - 즉, 출력 특성맥의 깊이와 너비가 줄어듦.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/3x3_patches_in_5x5_input.png" style="width:500px;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

- 경우 2: 패딩 있음, 보폭은 1.
    - 출력 특성맵의 깊이와 너비: `5x5`
    - 즉, 출력 특성맵의 깊이와 너비가 동일하게 유지됨.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/padding_of_5x5_input.png" style="width:500px;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

- 경우 3: 패딩 없음, 보폭은 2.
    - 출력 특성맵의 깊이와 너비: `2x2`
    - 즉, 출력 특성맵의 깊이와 너비가 보폭의 반비례해서 줄어듦.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/strides.png" style="width:500px;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

- 경우 4: 패딩 있음, 보폭은 2.
    - 이 경우는 굳이 사용할 필요 없음. 
        이유는 보폭이 1보다 크기에 출력 특성맵의 깊이와 너비가 어차피 보폭에 반비례해서 줄어들기 때문임.

### 맥스 풀링 연산

**맥스 풀링**(max-pooling) 연산은 일정 크기의 영역에서 최댓값만을 선택하여 
특성맵의 깊이와 너비를 일정 비율로 줄인다. 
예를 들어, 아래 맥스 풀링 층은 `2x2`영역에서 최댓값 하나만을 남기고 나머지는 버리며,
이 연산을 보폭 2만큼씩 이동하며 특성맵의 모든 영역(`높이x너비`)에 대해 실행한다. 

```python
layers.MaxPooling2D(pool_size=2)
```

따라서 만약 `x`가 `(26, 26, 32)` 모양의 3D 텐서이면
다음 맥스 풀링 층의 출력값은 `(13, 13, 32)` 모양의 3D 텐서가 된다.

```python
layers.MaxPooling2D(pool_size=2)(x)
```

맥스 풀링 층을 합성곱 층(`Conv2D`)와 함께 사용하는 이유는 두 가지이다. 

- 학습해야할 가중치 파라미터의 수 줄이기
- 상위에 위치한 층이 입력 특성맥의 보다 넓은 영역에 대한 정보를 얻도록 하기

아래 코드는 맥스 풀링 층을 사용하지 않는 경우 가중치 파라미터의 수가 엄청나게 증가함을 잘 보여준다. 

- 맥스 풀링 사용하는 경우: 104,202개
- 그렇지 않은 경우: 712,202개

In [5]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)

In [6]:
model_no_max_pool.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 128)       73856     
_________________________________________________________________
flatten_1 (Flatten)          (None, 61952)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                619530    
Total params: 712,202
Trainable params: 712,202
Non-trainable params: 0
_____________________________________________________

### 적절한 `kernel_size`, `stride`, `pool_size`

- `Conv2D` 층의 기본값: `kernel_size=3`, `stride=1`
- `MaxPooling2D` 층의 기본값: `pool_size=2`, `stride=2`

다른 설정 또는 최댓값 대신에 평균값을 사용하는 `AveragePooling2D`를 
활용할 수 있으나 케라스의 기본 설정이 가장 좋은 성능을 보인다.

## 8.2 합성곱 신경망 실전 활용 예제

### The relevance of deep learning for small-data problems

### Downloading the data

구글 코랩에서만 실행!

```python
from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c dogs-vs-cats

!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip
```

`kaggle` API 를 설치하거나 업그레이드 해야 할 수도 있음.
업그레이드는 다음과 같이 실행.

```python
!pip install --upgrade --force-reinstall --no-deps kaggle
```

**Copying images to training, validation, and test directories**

In [0]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

### Building your model

**Instantiating a small convnet for dogs vs. cats classification**

In [0]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [0]:
model.summary()

**Configuring the model for training**

In [0]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

### Data preprocessing

**Using `image_dataset_from_directory` to read images from directories**

In [0]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

**Instantiating a Dataset from a NumPy array**

In [0]:
import numpy as np
import tensorflow as tf
random_numbers = np.random.normal(size=(1000, 16))
dataset = tf.data.Dataset.from_tensor_slices(random_numbers)

**Iterating on a dataset**

In [0]:
for i, element in enumerate(dataset):
    print(element.shape)
    if i >= 2:
        break

**Batching a dataset**

In [0]:
batched_dataset = dataset.batch(32)
for i, element in enumerate(batched_dataset):
    print(element.shape)
    if i >= 2:
        break

**Applying a transformation to Dataset elements using `map()`**

In [0]:
reshaped_dataset = dataset.map(lambda x: tf.reshape(x, (4, 4)))
for i, element in enumerate(reshaped_dataset):
    print(element.shape)
    if i >= 2:
        break

**Displaying the shapes of the data and labels yielded by the Dataset**

In [0]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

**Fitting the model using a Dataset**

In [0]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

**Displaying curves of loss and accuracy during training**

In [0]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

**Evaluating the model on the test set**

In [0]:
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

### Using data augmentation

**Define a data augmentation stage to add to an image model**

In [0]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

**Displaying some randomly augmented training images**

In [0]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

**Defining a new convnet that includes image augmentation and dropout**

In [0]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

**Training the regularized convnet**

In [0]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks)

**Evaluating the model on the test set**

In [0]:
test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

## Leveraging a pretrained model

### Feature extraction with a pretrained model

**Instantiating the VGG16 convolutional base**

In [0]:
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))

In [0]:
conv_base.summary()

#### Fast feature extraction without data augmentation

**Extracting the VGG16 features and corresponding labels**

In [0]:
import numpy as np

def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels =  get_features_and_labels(train_dataset)
val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)

In [0]:
train_features.shape

**Defining and training the densely connected classifier**

In [0]:
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction.keras",
      save_best_only=True,
      monitor="val_loss")
]
history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

**Plotting the results**

In [0]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

#### Feature extraction together with data augmentation

**Instantiating and freeze the VGG16 convolutional base**

In [0]:
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
conv_base.trainable = False

**Printing the list of trainable weights before and after freezing**

In [0]:
conv_base.trainable = True
print("This is the number of trainable weights "
      "before freezing the conv base:", len(conv_base.trainable_weights))

In [0]:
conv_base.trainable = False
print("This is the number of trainable weights "
      "after freezing the conv base:", len(conv_base.trainable_weights))

**Adding data augmentation and a densely-connected classifier to the frozen convolutional base**

In [0]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [0]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)

**Evaluating the model on the test set**

In [0]:
test_model = keras.models.load_model(
    "feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

### Fine-tuning a pretrained model

In [0]:
conv_base.summary()

**Freezing all layers until the 4th from the last**

In [0]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

**Fine-tuning the model**

In [0]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [0]:
model = keras.models.load_model("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

## Chapter summary